
# Introduction to SQLAlchemy

SQLAlchemy is a popular Python library for working with relational databases. It provides both **ORM (Object Relational Mapping)** and **Core SQL Expression Language** functionalities. This allows developers to write Pythonic database code while retaining the ability to execute raw SQL when needed.

Key components of SQLAlchemy:
1. **Engine**: Manages the database connection.
2. **Session**: Handles transactions and queries.
3. **Declarative Base**: Allows defining database tables as Python classes.

In this lesson, we will:
1. Set up SQLAlchemy with a MySQL database.
2. Create tables using the Declarative Base.
3. Establish relationships (One-to-Many, Many-to-Many).
4. Perform CRUD (Create, Read, Update, Delete) operations.
5. Understand session management and cascading behaviors.
    


## Step 1: Setting Up MySQL Connection

To work with SQLAlchemy and MySQL:
1. Install the `pymysql` driver.
2. Connect to a MySQL database using SQLAlchemy's `create_engine`.

Replace the placeholders `<username>`, `<password>`, `<host>`, and `<database>` with your actual MySQL credentials.
    


## Step 2: Defining Tables with Relationships

We'll create:
1. A `User` table, which has a one-to-many relationship with `Post`.
2. A `Post` table, which has a many-to-many relationship with `Tag`.

### One-to-Many Relationship
A user can have multiple posts. This is modeled by linking the `Post` table to the `User` table using a foreign key.

### Many-to-Many Relationship
A post can have multiple tags, and a tag can belong to multiple posts. This is achieved using an **association table**.
    


## Step 3: Adding Data

We'll:
1. Create a user.
2. Add posts for the user.
3. Assign tags to the posts.

SQLAlchemy handles the relationships automatically when using ORM objects.
    


## Step 4: Querying Data

### Examples:
1. Fetch all posts for a user.
2. Fetch all tags for a post.
3. Fetch all posts associated with a specific tag.
    


## Step 5: Updating and Deleting Data

### Updating:
1. Update a post's title.
2. Remove a tag from a post.

### Deleting:
1. Delete a user (cascade deletes their posts).
    


## Summary

In this lesson, we covered:
1. Connecting SQLAlchemy to a MySQL database.
2. Defining tables with one-to-many and many-to-many relationships.
3. Performing CRUD operations on related data.
4. Understanding cascading behaviors in relationships.

SQLAlchemy's ORM provides a powerful, Pythonic way to interact with relational databases. Explore its advanced features like transactions, eager loading, and raw SQL queries to deepen your knowledge!
    

In [ ]:

# Install required libraries (Run this if not already installed)
!pip install sqlalchemy pymysql
    

In [ ]:

from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base, sessionmaker

# Replace these credentials with your MySQL details
DATABASE_URL = "mysql+pymysql://<username>:<password>@<host>/<database>"

# Create the engine
engine = create_engine(DATABASE_URL, echo=True)

# Define the Base class for the ORM models
Base = declarative_base()

# Create a session maker
Session = sessionmaker(bind=engine)
session = Session()
    

In [ ]:

from sqlalchemy import Column, Integer, String, ForeignKey, Table
from sqlalchemy.orm import relationship

# Association table for the Many-to-Many relationship
post_tag_table = Table(
    'post_tag',
    Base.metadata,
    Column('post_id', Integer, ForeignKey('posts.id'), primary_key=True),
    Column('tag_id', Integer, ForeignKey('tags.id'), primary_key=True)
)

# User model
class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(50), nullable=False)
    email = Column(String(100), unique=True, nullable=False)
    
    # One-to-Many relationship
    posts = relationship('Post', back_populates='user')
    
    def __repr__(self):
        return f"<User(name={self.name}, email={self.email})>"

# Post model
class Post(Base):
    __tablename__ = 'posts'
    
    id = Column(Integer, primary_key=True)
    title = Column(String(100), nullable=False)
    content = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'), nullable=False)
    
    # Relationships
    user = relationship('User', back_populates='posts')
    tags = relationship('Tag', secondary=post_tag_table, back_populates='posts')
    
    def __repr__(self):
        return f"<Post(title={self.title}, user={self.user.name})>"

# Tag model
class Tag(Base):
    __tablename__ = 'tags'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(50), unique=True, nullable=False)
    
    # Relationship
    posts = relationship('Post', secondary=post_tag_table, back_populates='tags')
    
    def __repr__(self):
        return f"<Tag(name={self.name})>"

# Create all tables in the database
Base.metadata.create_all(engine)
    

In [ ]:

# Create a user
user = User(name="Alice", email="alice@example.com")

# Create posts for the user
post1 = Post(title="First Post", content="Alice's first post", user=user)
post2 = Post(title="Second Post", content="Alice's second post", user=user)

# Create tags
tag1 = Tag(name="Introduction")
tag2 = Tag(name="SQLAlchemy")

# Associate tags with posts
post1.tags.extend([tag1, tag2])
post2.tags.append(tag2)

# Add and commit the data
session.add(user)
session.commit()

# Verify the data
print("Users:", session.query(User).all())
print("Posts:", session.query(Post).all())
print("Tags:", session.query(Tag).all())
    

In [ ]:

# Fetch all posts by a user
alice_posts = session.query(User).filter(User.name == "Alice").first().posts
print("Alice's Posts:", alice_posts)

# Fetch all tags for a specific post
post_tags = session.query(Post).filter(Post.title == "First Post").first().tags
print("Tags for 'First Post':", post_tags)

# Fetch all posts with a specific tag
tag_posts = session.query(Tag).filter(Tag.name == "SQLAlchemy").first().posts
print("Posts with tag 'SQLAlchemy':", tag_posts)
    

In [ ]:

# Update a post's title
post_to_update = session.query(Post).filter(Post.title == "First Post").first()
post_to_update.title = "Updated First Post"
session.commit()

# Remove a tag from a post
post_to_update.tags.remove(tag1)
session.commit()

# Delete a user and cascade to posts
user_to_delete = session.query(User).filter(User.name == "Alice").first()
session.delete(user_to_delete)
session.commit()

# Verify the changes
print("Remaining Users:", session.query(User).all())
print("Remaining Posts:", session.query(Post).all())
print("Remaining Tags:", session.query(Tag).all())
    